In [3]:
# python3
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [4]:
from ActualLatLong import *

In [12]:
path = "corr.csv"
df_corr = pd.read_csv(path)
del df_corr['Unnamed: 0']

""" add latlong back """
if 'lat' not in df_corr.columns:
    df_corr['lat'] = (df_corr['latlong'] // 1000.).astype(int)
    df_corr['long'] = (df_corr['latlong'] % 1000.).astype(int)

In [ ]:
""" get new lat_long """
new_lat_long = []

for y in range(0, -480, -1):
    for x in range(480):
        distance, angle = find_distance_angle(y, x)
        lat, long = displace(ORIGIN_LAT, ORIGIN_LONG, angle, distance)
        new_lat_long.append([-y, x, lat, long])

In [ ]:
ll_arr = np.array(new_lat_long)
test = pd.DataFrame(ll_arr, columns=["lat","long" ,"map_lat", "map_long"])
test['lat'] = test['lat'].astype(int)
test['long'] = test['long'].astype(int)
# test.to_csv("map_latlong.csv",index=False)

## Folium Playground

In [7]:
import folium
from folium import plugins
from folium.plugins import HeatMap

In [14]:
"""get centre of map"""
test = pd.read_csv("map_latlong.csv")
mid = test.loc[test['lat'] == 239].loc[test['long'] == 239]
mid_lat = mid['map_lat'].values[0]
mid_long = mid['map_long'].values[0]
m = folium.Map([mid_lat, mid_long], zoom_start=10)

In [20]:
cols = df_corr.columns.values[21]
df_corr['map_lat'] = test['map_lat']
df_corr['map_long'] = test['map_long']
temp = df_corr[['map_lat', 'map_long', cols]].copy()
temp = temp.fillna(0, inplace=False)
temp.isnull().sum()

map_lat            0
map_long           0
corr_station_20    0
dtype: int64

In [22]:
temp.max(axis=0)
med = temp.median(axis=0).corr_station_20
temp_filtered = temp.loc[temp['corr_station_20'] > med]

In [54]:
min = temp.quantile(q=0.999, axis=0).corr_station_20
temp_filtered = temp.loc[temp[cols] > min]
temp_filtered[cols] = temp_filtered[cols]

/Users/idawatibustan/.virtualenvs/py3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [58]:
m = folium.Map([mid_lat, mid_long], zoom_start=10)
max_amount = float(temp_filtered[cols].max())

data = np.array(temp_filtered).tolist()
HeatMap(data,
        min_opacity=0.9,
        max_val=max_amount,
        radius=7, blur=14,
        max_zoom=3
       ).add_to(m)
m.save('heatmap.html')

In [62]:
df_corr.columns.values[1:51]

array(['corr_station_0', 'corr_station_1', 'corr_station_2',
       'corr_station_3', 'corr_station_4', 'corr_station_5',
       'corr_station_6', 'corr_station_7', 'corr_station_8',
       'corr_station_9', 'corr_station_10', 'corr_station_11',
       'corr_station_12', 'corr_station_13', 'corr_station_14',
       'corr_station_15', 'corr_station_16', 'corr_station_17',
       'corr_station_18', 'corr_station_19', 'corr_station_20',
       'corr_station_21', 'corr_station_22', 'corr_station_23',
       'corr_station_24', 'corr_station_25', 'corr_station_26',
       'corr_station_27', 'corr_station_28', 'corr_station_29',
       'corr_station_30', 'corr_station_31', 'corr_station_32',
       'corr_station_33', 'corr_station_34', 'corr_station_35',
       'corr_station_36', 'corr_station_37', 'corr_station_38',
       'corr_station_39', 'corr_station_40', 'corr_station_41',
       'corr_station_42', 'corr_station_43', 'corr_station_44',
       'corr_station_45', 'corr_station_46', 'corr

In [64]:
"""generate html visualization on map"""
for cols in df_corr.columns.values[1:51]:
    print(cols)
    temp = df_corr[['map_lat', 'map_long', cols]].copy()
    temp = temp.fillna(0, inplace=False)
    temp.isnull().sum()

    min_amount = temp.quantile(q=0.999, axis=0)[cols]
    temp_filtered = temp.loc[temp[cols] > min_amount]
    temp_filtered[cols] = temp_filtered[cols]*1000
    max_amount = float(temp_filtered[cols].max())
    
    m = folium.Map([mid_lat, mid_long], zoom_start=10)
    data = np.array(temp_filtered).tolist()
    HeatMap(data,
            min_opacity=0.9,
            max_val=max_amount,
            radius=7, blur=14,
            max_zoom=3
           ).add_to(m)
    m.save('heatmaps/heatmap_'+cols+'.html')

corr_station_0
corr_station_1


/Users/idawatibustan/.virtualenvs/py3/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


corr_station_2
corr_station_3
corr_station_4
corr_station_5
corr_station_6
corr_station_7
corr_station_8
corr_station_9
corr_station_10
corr_station_11
corr_station_12
corr_station_13
corr_station_14
corr_station_15
corr_station_16
corr_station_17
corr_station_18
corr_station_19
corr_station_20
corr_station_21
corr_station_22
corr_station_23
corr_station_24
corr_station_25
corr_station_26
corr_station_27
corr_station_28
corr_station_29
corr_station_30
corr_station_31
corr_station_32
corr_station_33
corr_station_34
corr_station_35
corr_station_36
corr_station_37
corr_station_38
corr_station_39
corr_station_40
corr_station_41
corr_station_42
corr_station_43
corr_station_44
corr_station_45
corr_station_46
corr_station_47
corr_station_48
corr_station_49
